In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
from datetime import datetime

In [5]:
# !pip download scikit-maad==1.4.1 -d /kaggle/working/mysitepackages/sklearn_maad_pkg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 67.8 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 69.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 73.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 37.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 85.5 MB/s eta

In [25]:
#!pip download ffmpeg-python==0.2.0 -d /kaggle/working/mysitepackages/ffmpeg_pkg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 8.2 MB/s eta 0:00:00a 0:00:01
Saved ./ffmpeg_python-0.2.0-py3-none-any.whl
Saved ./future-1.0.0-py3-none-any.whl
Successfully downloaded ffmpeg-python future


In [29]:
"""from zipfile import ZipFile
import os

dirName = '/kaggle/working/mysitepackages'
zipName = 'ffmpeg_pkg.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

In [1]:
!pip install ffmpeg-python --no-index --find-links=file:///kaggle/input/ffmpeg-pkg/kaggle/working

Looking in links: file:///kaggle/input/ffmpeg-pkg/kaggle/working
Processing /kaggle/input/ffmpeg-pkg/kaggle/working/ffmpeg_python-0.2.0-py3-none-any.whl


In [2]:
!pip install scikit-maad --no-index --find-links=file:///kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg

Looking in links: file:///kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg
Processing /kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg/scikit_maad-1.4.1-py3-none-any.whl
Processing /kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg/resampy-0.4.3-py3-none-any.whl (from scikit-maad)


In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from maad import sound, rois, features
from maad.util import (
   power2dB, plot2d, format_features, read_audacity_annot,
   overlay_rois, overlay_centroid
   )

In [16]:
audio_duration = 5

# Function to extract features from audio file
def extract_features(file_path, offset=0.0, duration=5.0):
    # Load audio file
    audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
    """total_duration = librosa.get_duration(y=audio, sr=sample_rate)
    num_segments = int(total_duration // audio_duration)"""
    # Extract features using Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40, n_fft=1024)
    # Flatten the features into a 1D array
    flattened_features = np.mean(mfccs.T, axis=0)
    return flattened_features

In [17]:
def audio_classification(file_path, offset=0.0, duration=5.0):
    extracted_features = extract_features(file_path, offset=offset, duration=duration).reshape(1, -1)
    y_predict = random_forest_model.predict_proba(extracted_features)
    
    return y_predict

In [8]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [9]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)

In [10]:
bird_cols = list(meta_data['primary_label'].unique().sort())

In [11]:
test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes'
# test_soundscapes = '/kaggle/input/birdclef-2024/test_soundscapes'

filenames_with_path = glob.glob(f"{test_soundscapes}/*.ogg")
# filenames = [('soundscape_' + os.path.basename(filename)) for filename in filenames_with_path]
filenames = [os.path.basename(filename) for filename in filenames_with_path]

print(len(filenames))

8444


In [ ]:
all_durations = []
for i in range(len(filenames_with_path)):
    audio, sample_rate = librosa.load(path='/kaggle/input/birdclef-2024/unlabeled_soundscapes/1627554722.ogg')
    duration = len(audio) / sample_rate
    all_durations.append(round(duration))
    
print(len(all_durations))

In [ ]:
"""audio, sample_rate = librosa.load(path=test_soundscapes, offset=0.0, duration=5.0)
audio_full, sample_rate_full = librosa.load(path=test_soundscapes)"""

In [ ]:
# Audio(audio, rate=sample_rate)

In [ ]:
"""duration = len(audio) / sample_rate
time = np.arange(0, duration, 1/sample_rate)
plt.figure(figsize=(30, 4))
plt.plot(time, audio, color='blue')
plt.title('Audio Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plot = plt.show()"""

In [ ]:
"""n_fft = 500  # Number of FFT points 2048
hop_length = 50  # Hop length for STFT 512
stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
# Convert the magnitude spectrogram to decibels (log scale)
spectrogram = librosa.amplitude_to_db(np.abs(stft))
# Plot the spectrogram
plt.figure(figsize=(30, 6))
librosa.display.specshow(spectrogram, sr=sample_rate, hop_length=hop_length, x_axis='time', y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plot = plt.show()"""

In [12]:
def audio_waveframe(file_path, offset=0.0, duration=5.0):
    # Load the audio file
    audio_data, sampling_rate = librosa.load(file_path, offset=offset, duration=duration)
    # Calculate the duration of the audio file
    duration = len(audio_data) / sampling_rate
    # Create a time array for plotting
    time = np.arange(0, duration, 1/sampling_rate)
    # Plot the waveform
    plt.figure(figsize=(30, 4))
    plt.plot(time, audio_data, color='blue')
    plt.title('Audio Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plot = plt.show()
    return plot

def spectrogram(file_path, offset=0.0, duration=5.0):
    # Compute the short-time Fourier transform (STFT)
    n_fft = 500  # Number of FFT points 2048
    hop_length = 50  # Hop length for STFT 512
    audio_data, sampling_rate = librosa.load(file_path, offset=offset, duration=duration)
    stft = librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length)
    # Convert the magnitude spectrogram to decibels (log scale)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    # Plot the spectrogram
    plt.figure(figsize=(30, 6))
    librosa.display.specshow(spectrogram, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='linear')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plot = plt.show()
    return plot

def audio_analysis(file_path, offset=0.0, duration=5.0):
    aw = audio_waveframe(file_path, offset, duration)
    spg = spectrogram(file_path, offset, duration)
    return aw, spg

In [ ]:
"""test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/100350238.ogg'
for j in range(0,241,5):
    predicted = audio_classification(test_soundscapes, offset=float(j), duration=5.0)
    filename_noext = test_soundscapes.replace('.ogg','')
    df['row_id'] = f'{filename_noext}_{j}'
    df[bird_cols] = predicted
    submission_df = pd.concat([submission_df,df]).reset_index(drop=True)"""

In [53]:
sample_submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit = pd.DataFrame(columns=sample_submit.columns, dtype="object")

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1


In [35]:
submit.columns

Index(['row_id', 'asbfly', 'ashdro1', 'ashpri1', 'ashwoo2', 'asikoe2',
       'asiope1', 'aspfly1', 'aspswi1', 'barfly1',
       ...
       'whbwoo2', 'whcbar1', 'whiter2', 'whrmun', 'whtkin2', 'woosan',
       'wynlau1', 'yebbab1', 'yebbul3', 'zitcis1'],
      dtype='object', length=183)

In [49]:
# 35
print(filenames_with_path[122])
print(filenames[122])

/kaggle/input/birdclef-2024/unlabeled_soundscapes/1627554722.ogg
1627554722.ogg


In [18]:
predicted = audio_classification('/kaggle/input/birdclef-2024/unlabeled_soundscapes/1627554722.ogg'
                                  , offset=90.0, duration=5.0)

/tmp/ipykernel_33/1830216342.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=0
  warnings.warn(


In [27]:
audio, sample_rate = librosa.load(path='/kaggle/input/birdclef-2024/unlabeled_soundscapes/1627554722.ogg'
                                  , offset=85.0, duration=5.0)

In [50]:
audio, sample_rate = librosa.load(path='/kaggle/input/birdclef-2024/unlabeled_soundscapes/1627554722.ogg')

In [51]:
duration = len(audio) / sample_rate
round(duration)

36

In [52]:
Audio(audio, rate=sample_rate)

In [ ]:
audio_analysis(file_path, offset=0.0, duration=5.0)

In [54]:
submission_df = pd.DataFrame(columns=sample_submit.columns, dtype="object")
print("submission file shape at start", submission_df.shape)
start = datetime.now()
print("inference started at : ", start)

audio, sample_rate = librosa.load(path=filenames_with_path[122])
duration = round(len(audio) / sample_rate)

for j in range(5,duration,5):
    now = datetime.now()
    offset = j-5
    predicted = audio_classification(filenames_with_path[122], offset=float(offset), duration=5.0)
    filename_noext = filenames[122].replace('.ogg','')
    # print(f'{filename_noext}_{j}')
    submit['row_id'] = filename_noext+'_'+str(j)
    submit[bird_cols] = predicted.round(5)
    # print(submit['row_id'])
    submission_df = pd.concat([submission_df,submit], axis=0, join='inner').reset_index(drop=True)
    later = datetime.now()
    difference = (later - now).total_seconds()
        
    print("processing of chunk {0} finished in {1} seconds".format(str(j), difference))
    
end = datetime.now()
ended_in = (end - start).total_seconds()
print("inference finished in : ", ended_in)

submission file shape at start (0, 183)
inference started at :  2024-04-15 18:09:32.191689


/tmp/ipykernel_33/904238.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submission_df = pd.concat([submission_df,submit], axis=0, join='inner').reset_index(drop=True)


processing of chunk 5 finished in 0.133762 seconds
processing of chunk 10 finished in 0.085884 seconds
processing of chunk 15 finished in 0.10451 seconds
processing of chunk 20 finished in 0.101641 seconds
processing of chunk 25 finished in 0.096713 seconds
processing of chunk 30 finished in 0.099618 seconds
processing of chunk 35 finished in 0.104454 seconds
inference finished in :  0.800876


In [15]:
# Test = True

# submission_df = pd.DataFrame(columns=['row_id']+bird_cols)
submission_df = pd.DataFrame(columns=sample_submit.columns, dtype="object")
print("submission file shape at start", submission_df.shape)
start = datetime.now()
print("inference started at : ", start)

for i in range(len(filenames_with_path)):
    now = datetime.now()
    for j in range(5,241,5):
        offset = j-5
        predicted = audio_classification(filenames_with_path[i], offset=float(offset), duration=5.0)
        filename_noext = filenames[i].replace('.ogg','')
        # print(f'{filename_noext}_{j}')
        submit['row_id'] = filename_noext+'_'+str(j)
        submit[bird_cols] = predicted.round(5)
        # print(submit['row_id'])
        submission_df = pd.concat([submission_df,submit]).reset_index(drop=True)
        later = datetime.now()
        difference = (later - now).total_seconds()
        
    print("processing of file {0} finished in {1} seconds".format(i, difference))
    
    if i == 1199:
        end = datetime.now()
        difference = (end - start).total_seconds()
        print("processing of 1200 files finished in {0} seconds".format(difference))
        print("submission file shape at end", submission_df.shape)
        break

submission file shape at start (0, 183)
inference started at :  2024-04-14 17:18:42.745931


/tmp/ipykernel_33/871938763.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submission_df = pd.concat([submission_df,submit]).reset_index(drop=True)


processing of file 0 finished in 5.253547 seconds
processing of file 1 finished in 5.081197 seconds
processing of file 2 finished in 5.265507 seconds
processing of file 3 finished in 5.364062 seconds
processing of file 4 finished in 5.141072 seconds
processing of file 5 finished in 5.362861 seconds
processing of file 6 finished in 5.158551 seconds
processing of file 7 finished in 5.34755 seconds
processing of file 8 finished in 5.145639 seconds
processing of file 9 finished in 5.238339 seconds
processing of file 10 finished in 5.496854 seconds
processing of file 11 finished in 5.385699 seconds
processing of file 12 finished in 5.339134 seconds
processing of file 13 finished in 5.268279 seconds
processing of file 14 finished in 5.175052 seconds
processing of file 15 finished in 5.366394 seconds
processing of file 16 finished in 5.817814 seconds
processing of file 17 finished in 5.241372 seconds
processing of file 18 finished in 5.422408 seconds
processing of file 19 finished in 5.169877

/tmp/ipykernel_33/2878017421.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/tmp/ipykernel_33/2878017421.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native

processing of file 35 finished in 11.708338 seconds
processing of file 36 finished in 5.335014 seconds
processing of file 37 finished in 5.579974 seconds
processing of file 38 finished in 5.677987 seconds
processing of file 39 finished in 5.936868 seconds
processing of file 40 finished in 5.779068 seconds
processing of file 41 finished in 5.819948 seconds
processing of file 42 finished in 5.839958 seconds
processing of file 43 finished in 5.713673 seconds
processing of file 44 finished in 5.925099 seconds
processing of file 45 finished in 5.56238 seconds
processing of file 46 finished in 5.969046 seconds
processing of file 47 finished in 5.731122 seconds
processing of file 48 finished in 5.855621 seconds
processing of file 49 finished in 5.890503 seconds
processing of file 50 finished in 5.559762 seconds
processing of file 51 finished in 5.944034 seconds
processing of file 52 finished in 5.779939 seconds
processing of file 53 finished in 5.956516 seconds
processing of file 54 finished 

...

processing of file 1033 finished in 14.724783 seconds
processing of file 1034 finished in 9.389725 seconds
processing of file 1035 finished in 9.371684 seconds
processing of file 1036 finished in 9.18386 seconds
processing of file 1037 finished in 9.511582 seconds
processing of file 1038 finished in 9.432856 seconds
processing of file 1039 finished in 9.404719 seconds
processing of file 1040 finished in 9.447966 seconds
processing of file 1041 finished in 9.208936 seconds
processing of file 1042 finished in 9.284543 seconds
processing of file 1043 finished in 9.405682 seconds
processing of file 1044 finished in 9.466065 seconds
processing of file 1045 finished in 9.259656 seconds
processing of file 1046 finished in 9.526279 seconds
processing of file 1047 finished in 9.478072 seconds
processing of file 1048 finished in 9.420965 seconds
processing of file 1049 finished in 9.229918 seconds
processing of file 1050 finished in 9.443532 seconds
processing of file 1051 finished in 9.151182 s

/tmp/ipykernel_33/2878017421.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/tmp/ipykernel_33/2878017421.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native

processing of file 1098 finished in 15.596024 seconds
processing of file 1099 finished in 9.434276 seconds
processing of file 1100 finished in 9.596496 seconds
processing of file 1101 finished in 9.486942 seconds
processing of file 1102 finished in 9.679756 seconds
processing of file 1103 finished in 9.497355 seconds
processing of file 1104 finished in 9.680866 seconds
processing of file 1105 finished in 9.642114 seconds
processing of file 1106 finished in 9.464792 seconds
processing of file 1107 finished in 9.648156 seconds
processing of file 1108 finished in 9.488579 seconds
processing of file 1109 finished in 9.714512 seconds
processing of file 1110 finished in 9.696915 seconds
processing of file 1111 finished in 9.542764 seconds
processing of file 1112 finished in 9.629704 seconds
processing of file 1113 finished in 9.642159 seconds
processing of file 1114 finished in 9.737518 seconds
processing of file 1115 finished in 9.527949 seconds
processing of file 1116 finished in 9.477407 

/tmp/ipykernel_33/2878017421.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/tmp/ipykernel_33/2878017421.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native

processing of file 1156 finished in 15.54992 seconds
processing of file 1157 finished in 9.826846 seconds
processing of file 1158 finished in 9.811227 seconds
processing of file 1159 finished in 9.76044 seconds
processing of file 1160 finished in 9.808201 seconds
processing of file 1161 finished in 9.607952 seconds
processing of file 1162 finished in 9.616093 seconds
processing of file 1163 finished in 9.851724 seconds
processing of file 1164 finished in 9.87779 seconds
processing of file 1165 finished in 9.644815 seconds
processing of file 1166 finished in 9.854163 seconds
processing of file 1167 finished in 9.847694 seconds
processing of file 1168 finished in 9.872864 seconds
processing of file 1169 finished in 9.755634 seconds
processing of file 1170 finished in 9.879609 seconds
processing of file 1171 finished in 9.661947 seconds
processing of file 1172 finished in 9.877853 seconds
processing of file 1173 finished in 9.669865 seconds
processing of file 1174 finished in 9.772284 sec

In [ ]:
# submission_df.sum(axis=1, numeric_only=True)

In [55]:
submission_df

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,NaN,0.00,0.02,0.01,0.0,0.01,0.0,0.04,0.0,0.0,...,0.00,0.01,0.00,0.02,0.01,0.01,0.00,0.01,0.01,0.01
1,1627554722_10,0.01,0.02,0.00,0.0,0.00,0.0,0.02,0.0,0.0,...,0.01,0.01,0.02,0.01,0.00,0.00,0.00,0.00,0.02,0.00
2,1627554722_15,0.01,0.04,0.00,0.0,0.00,0.0,0.02,0.0,0.0,...,0.00,0.02,0.01,0.01,0.00,0.00,0.01,0.00,0.01,0.00
3,1627554722_20,0.00,0.01,0.00,0.0,0.01,0.0,0.03,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01
4,1627554722_25,0.01,0.03,0.00,0.0,0.00,0.0,0.01,0.0,0.0,...,0.00,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00
5,1627554722_30,0.00,0.01,0.00,0.0,0.00,0.0,0.02,0.0,0.0,...,0.00,0.01,0.01,0.02,0.00,0.01,0.01,0.00,0.00,0.00
6,1627554722_35,0.00,0.02,0.00,0.0,0.00,0.0,0.02,0.0,0.0,...,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00


In [ ]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submission_df.columns)
#submission_df = submission_df[sample_submission.columns]

In [ ]:
submission_df.to_csv('submission.csv', index=False)